In [1]:
# !pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import mediapipe as mp
import cv2
import numpy as np  

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [4]:
VIDEO_PATH = 'lean.avi'
EXPORT_PATH = 'annotations/deadlift/lean.csv'
MODEL_PATH = 'models/deadlift/lean.pkl'

1.1 Save Video

In [5]:
cap = cv2.VideoCapture(0)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter(VIDEO_PATH, cv2.VideoWriter_fourcc('P', 'I', 'M', '1'), fps, (int(width), int(height)))

cv2.VideoCapture()

while cap.isOpened():
    ret,frame = cap.read()
    
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    try:
        cv2.imshow('Press', frame)
        videoWriter.write(frame)
    except Exception as e:
        break

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()


2. Capture Landmars and Export to CSV

In [3]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [4]:
landmarks = ['class']
for val in range(1,33+1):
    landmarks += ['x{}'.format(val),'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [ ]:
with open(EXPORT_PATH, mode='w', newline='') as f:
    csv_writer = csv.writer(f,delimiter = ',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [7]:
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist() #Converted to a list
        # print(keypoints)
        keypoints.insert(0,action)
        
        with open(EXPORT_PATH, mode='a', newline='') as f:
            csv_writer = csv.writer(f,delimiter = ',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)
        pass

In [ ]:
cap = cv2.VideoCapture(VIDEO_PATH) #name of the video
#Initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()

        #Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make detection
        results = pose.process(image)

        #Recolor back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #Render detections
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                )

        k = cv2.waitKey(1)
        if k== ord('a'):
            export_landmark(results,'left')
        elif k== ord('s'):
            export_landmark(results,'neutral')
        elif k== ord('d'):
            export_landmark(results,'right')
        
        cv2.imshow('Mediapipe Feed',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


3. Train Custom Model Using Scikit Learn

3.1 Read in Collected Data and Process

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [25]:
df = pd.read_csv('coords.csv')

In [26]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,up,0.686481,0.148661,-0.351914,0.999988,0.693672,0.125929,-0.338640,0.999954,0.697559,...,0.195440,0.887369,0.735034,0.968800,0.056470,0.979596,0.634200,0.962292,0.073730,0.982735
1,up,0.688938,0.157763,-0.348684,0.999989,0.695556,0.137402,-0.335608,0.999952,0.699117,...,0.185030,0.891953,0.735212,0.967485,0.059648,0.984818,0.636459,0.964553,0.057357,0.983184
2,up,0.689142,0.157902,-0.348818,0.999989,0.695772,0.137448,-0.335504,0.999952,0.699313,...,0.183829,0.891244,0.735275,0.967463,0.059115,0.984989,0.636578,0.964563,0.056552,0.983107
3,up,0.689371,0.158029,-0.349248,0.999989,0.695997,0.137495,-0.335827,0.999952,0.699504,...,0.183558,0.891693,0.735323,0.967462,0.059732,0.985280,0.636522,0.964574,0.055821,0.983226
4,up,0.689636,0.158105,-0.346876,0.999989,0.696237,0.137531,-0.333257,0.999951,0.699702,...,0.183308,0.892057,0.735461,0.967453,0.060765,0.985520,0.636501,0.964581,0.055437,0.983294


In [27]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
141,up,0.687948,0.350127,-0.676594,0.999994,0.694703,0.327855,-0.659558,0.999981,0.698712,...,-0.100999,0.875294,0.738784,0.975694,-0.285752,0.976663,0.634489,0.972725,-0.272543,0.976513
142,up,0.689459,0.385049,-0.700267,0.999993,0.695962,0.364850,-0.684117,0.999981,0.699657,...,-0.106889,0.866016,0.739247,0.974905,-0.296650,0.966665,0.630477,0.969722,-0.271714,0.968335
143,up,0.689947,0.418463,-0.760781,0.999993,0.696464,0.399773,-0.741264,0.999980,0.700218,...,-0.089544,0.852271,0.739445,0.973817,-0.298148,0.952491,0.629650,0.968418,-0.264526,0.958548
144,up,0.689914,0.434866,-0.742952,0.999992,0.696479,0.416724,-0.724676,0.999979,0.700303,...,-0.085563,0.842661,0.740107,0.972228,-0.283973,0.939931,0.629419,0.967626,-0.256425,0.953271
145,up,0.690106,0.456903,-0.735617,0.999990,0.696609,0.438140,-0.717469,0.999979,0.700414,...,-0.086030,0.823201,0.740117,0.969551,-0.271345,0.921514,0.630092,0.965973,-0.251966,0.943396


In [28]:
df[df['class'] == 'up']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,up,0.686481,0.148661,-0.351914,0.999988,0.693672,0.125929,-0.338640,0.999954,0.697559,...,0.195440,0.887369,0.735034,0.968800,0.056470,0.979596,0.634200,0.962292,0.073730,0.982735
1,up,0.688938,0.157763,-0.348684,0.999989,0.695556,0.137402,-0.335608,0.999952,0.699117,...,0.185030,0.891953,0.735212,0.967485,0.059648,0.984818,0.636459,0.964553,0.057357,0.983184
2,up,0.689142,0.157902,-0.348818,0.999989,0.695772,0.137448,-0.335504,0.999952,0.699313,...,0.183829,0.891244,0.735275,0.967463,0.059115,0.984989,0.636578,0.964563,0.056552,0.983107
3,up,0.689371,0.158029,-0.349248,0.999989,0.695997,0.137495,-0.335827,0.999952,0.699504,...,0.183558,0.891693,0.735323,0.967462,0.059732,0.985280,0.636522,0.964574,0.055821,0.983226
4,up,0.689636,0.158105,-0.346876,0.999989,0.696237,0.137531,-0.333257,0.999951,0.699702,...,0.183308,0.892057,0.735461,0.967453,0.060765,0.985520,0.636501,0.964581,0.055437,0.983294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,up,0.687948,0.350127,-0.676594,0.999994,0.694703,0.327855,-0.659558,0.999981,0.698712,...,-0.100999,0.875294,0.738784,0.975694,-0.285752,0.976663,0.634489,0.972725,-0.272543,0.976513
142,up,0.689459,0.385049,-0.700267,0.999993,0.695962,0.364850,-0.684117,0.999981,0.699657,...,-0.106889,0.866016,0.739247,0.974905,-0.296650,0.966665,0.630477,0.969722,-0.271714,0.968335
143,up,0.689947,0.418463,-0.760781,0.999993,0.696464,0.399773,-0.741264,0.999980,0.700218,...,-0.089544,0.852271,0.739445,0.973817,-0.298148,0.952491,0.629650,0.968418,-0.264526,0.958548
144,up,0.689914,0.434866,-0.742952,0.999992,0.696479,0.416724,-0.724676,0.999979,0.700303,...,-0.085563,0.842661,0.740107,0.972228,-0.283973,0.939931,0.629419,0.967626,-0.256425,0.953271


In [29]:
X = df.drop('class', axis=1) #Features
Y = df['class'] #Target value

In [30]:
X_train, X_test, y_train,y_test = train_test_split(X,Y,test_size = 0.3, random_state = 1234)

In [26]:
X

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,0.686481,0.148661,-0.351914,0.999988,0.693672,0.125929,-0.338640,0.999954,0.697559,0.125748,...,0.195440,0.887369,0.735034,0.968800,0.056470,0.979596,0.634200,0.962292,0.073730,0.982735
1,0.688938,0.157763,-0.348684,0.999989,0.695556,0.137402,-0.335608,0.999952,0.699117,0.137053,...,0.185030,0.891953,0.735212,0.967485,0.059648,0.984818,0.636459,0.964553,0.057357,0.983184
2,0.689142,0.157902,-0.348818,0.999989,0.695772,0.137448,-0.335504,0.999952,0.699313,0.137129,...,0.183829,0.891244,0.735275,0.967463,0.059115,0.984989,0.636578,0.964563,0.056552,0.983107
3,0.689371,0.158029,-0.349248,0.999989,0.695997,0.137495,-0.335827,0.999952,0.699504,0.137167,...,0.183558,0.891693,0.735323,0.967462,0.059732,0.985280,0.636522,0.964574,0.055821,0.983226
4,0.689636,0.158105,-0.346876,0.999989,0.696237,0.137531,-0.333257,0.999951,0.699702,0.137209,...,0.183308,0.892057,0.735461,0.967453,0.060765,0.985520,0.636501,0.964581,0.055437,0.983294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,0.686107,0.494564,-0.650963,0.999987,0.692495,0.476825,-0.637222,0.999980,0.696505,0.475535,...,-0.075127,0.827533,0.737980,0.968581,-0.234898,0.940578,0.628790,0.960369,-0.222426,0.956996
123,0.686039,0.494472,-0.652628,0.999987,0.692441,0.476796,-0.638969,0.999980,0.696460,0.475519,...,-0.075903,0.829990,0.738036,0.968629,-0.236920,0.942006,0.628788,0.960535,-0.224383,0.957898
124,0.685979,0.494424,-0.652721,0.999987,0.692402,0.476766,-0.639155,0.999980,0.696427,0.475489,...,-0.077804,0.832161,0.738098,0.968663,-0.240544,0.943236,0.628738,0.960594,-0.226933,0.958620
125,0.685938,0.494382,-0.663384,0.999987,0.692378,0.476775,-0.649429,0.999980,0.696407,0.475518,...,-0.079943,0.835241,0.738168,0.968772,-0.245082,0.945252,0.628571,0.960610,-0.231201,0.959665


3.2 Train Machine Learning Classification Model

In [17]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [21]:
pipelines = {
    'lr':make_pipeline(StandardScaler(),LogisticRegression()),
    'rc':make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier()),
}

In [22]:
fit_models = {}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

NameError: name 'X_train' is not defined

In [30]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

3.3 Evaluate and Serialize Model


In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score #Accuracy metrics
import pickle

In [19]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test.values,yhat),
            precision_score(y_test.values,yhat,average='binary',pos_label='up'),
            recall_score(y_test.values,yhat,average='binary',pos_label='up'))

NameError: name 'fit_models' is not defined

In [40]:
yhat = fit_models['rf'].predict(X_test)

In [41]:
yhat[:10]

array(['down', 'up', 'down', 'up', 'up', 'down', 'up', 'down', 'up',
       'down'], dtype=object)

In [42]:
y_test

103    down
63       up
109    down
42       up
35       up
120    down
55       up
111    down
22       up
97     down
62       up
112    down
88       up
8        up
6        up
98     down
64       up
57       up
32       up
91       up
94       up
27       up
77       up
79       up
59       up
78       up
1        up
29       up
123    down
9        up
74       up
84       up
40       up
17       up
48     down
61       up
21       up
33       up
110    down
Name: class, dtype: object

In [43]:
with open('deadlift.pkl','wb') as f:  #dumping the model
    pickle.dump(fit_models['rf'],f)

4. Make Detections with Model

In [31]:
with open('deadlift.pkl','rb') as f:
    model = pickle.load(f)

In [ ]:
# X = pd.DataFrame([row], columns=landmarks[1:])

In [34]:
cap = cv2.VideoCapture(0)
counter = 0
current_stage = ''
#Initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        #Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make detections
        results = pose.process(image)

        #Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #Extract landmarks
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                )

        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row],columns=landmarks[1:])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class,body_language_prob)

            if body_language_class == 'down' and body_language_prob[body_language_prob.argmax()] >= 0.7:
                current_stage = 'down'
            elif current_stage == 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] >= 0.7:
                current_stage = 'up'
                counter += 1
                print(current_stage,counter)

            #Get status box
            cv2.rectangle(image,(0,0),(225,73),(245,117,16),-1)

            #Display Class
            cv2.putText(image,'CLASS',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,body_language_class.split(' ')[0],(90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            #Display Probability
            cv2.putText(image,'PROB',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(round(body_language_prob[np.argmax(body_language_prob)],2)),(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            #Display Counter
            cv2.putText(image,'COUNTER',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(counter),(175,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        except Exception as e:
            print(e)
            pass
        
        cv2.imshow('Raw Webcam Feed',image)

        if(cv2.waitKey(10) & 0xFF == ord('q')):
            break

cap.release()
cv2.destroyAllWindows()


'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute